In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as st
import pickle
import utils
import moran_auto
import gmm
import kmeans_utils
import figure_functions
from sklearn.preprocessing import MinMaxScaler
from sklearn import mixture
from sklearn.cluster import KMeans
import csv
from scipy.misc import imread
from collections import defaultdict
from matplotlib.patches import Ellipse
from matplotlib.patches import Polygon
import glob
from pandas.tseries.holiday import USFederalHolidayCalendar
import datetime
import gmplot
import matplotlib.cm as cm
import run_all

In [2]:
curr_dir = os.getcwd()
data_path = curr_dir + '/../data'
fig_path = curr_dir + '/../figs'
results_path = curr_dir + '/../results'
animation_path = curr_dir + '/../animation'
belltown_path = data_path + '/Belltown_Hour'
commcore_path = data_path + '/CommercialCore_Hour'
pikepine_path = data_path + '/PikePine_Hour'

In [3]:
element_keys, gps_loc, loads, park_data, N, P, idx_to_day_hour, day_hour_to_idx = utils.load_data_new(data_path, belltown_path)

In [4]:
time = 8

In [5]:
run_all.run_figures(loads, gps_loc, N, P, fig_path, animation_path, time)

In [ ]:
figure_functions.spatial_heterogeneity(loads, time, N, fig_path)

In [ ]:
bins = range(N)

mask = ~np.isnan(loads[:, time])
loads = loads[mask]

# Getting the load data for a specific time to plot for each block key.
counts = loads[:, time] * 100

In [ ]:
loads.shape

In [ ]:
idx_to_day_hour

In [ ]:
gps_loc, belltown_avg_loads, park_data, N, P, idx_to_day_hour, day_hour_to_idx, element_keys_belltown = load_data(data_path, belltown_path)
gps_loc, commcore_avg_loads, park_data, N, P, idx_to_day_hour, day_hour_to_idx, element_keys_commcore = load_data(data_path, commcore_path)
gps_loc, pikepine_avg_loads, park_data, N, P, idx_to_day_hour, day_hour_to_idx, element_keys_pikepine = load_data(data_path, pikepine_path)

In [ ]:
gps_loc, avg_loads, park_data, N, P, idx_to_day_hour, day_hour_to_idx, element_keys = load_data(data_path, belltown_path)

In [ ]:
adjacency = np.zeros((len(block_keys), len(block_keys)))
N = len(block_keys)

for i in xrange(N):
    if block_keys[i] in north:
        zone1 = 'north'
    elif block_keys[i] in south:
        zone1 = 'south'
    else:
        print block_keys[i]
    for j in xrange(N):
        if i == j:
            continue
        
        if block_keys[j] in north:
            zone2 = 'north'
        elif block_keys[j] in south:
            zone2 = 'south'
        else:
            print block_keys[j]
        
        if zone1 == zone2:
            adjacency[i, j] = 1
        else:
            pass

In [ ]:
fig_path = r'/Users/tfiez/Dropbox/2017Summer/tex/2018ACC/paper/figs'

In [ ]:
for i in range(avg_loads.shape[1]):
    print len(np.where(avg_loads[:, i] == 0)[0])

In [ ]:
fig, ax = figure_functions.contour_plot(loads=avg_loads, gps_loc=gps_loc, time=51,
                                        title='', N=N, filename='contour1.png',
                                        fig_path=fig_path, contours=10)

fig, ax = figure_functions.mixture_plot(loads=avg_loads, gps_loc=gps_loc, 
                                            times=[51], N=N, fig_path=fig_path, 
                                            shape=(1,1), filename='mixture1.png',
                                            title='')

plt.show()

In [ ]:
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)
c = a*b

In [ ]:
c

In [ ]:
fig, ax = figure_functions.temporal_day_plots(loads=avg_loads*100, P=P, fig_path=r'/Users/tfiez/Dropbox/2017Summer/tex/2018ACC/paper/figs')

In [ ]:
time=50
time1 = 50
time2 = 58

In [ ]:
len(new_park_data.loc[(new_park_data['Day'] == 4) & (new_park_data['Hour'] == 17)]['Date'].unique().tolist())

In [ ]:
means = pickle.load(open('means.pkl', 'rb'))

In [ ]:
distances, centroids = kmeans_utils.get_distances(means)
all_time_points = kmeans_utils.get_centroid_circle_paths(distances, centroids)

In [ ]:
fig, ax = figure_functions.centroid_radius(centroids, all_time_points, gps_loc,
                                           times=[time2], fig_path=fig_path, shape=(1,1))
plt.show()

fig, ax = figure_functions.centroid_plots(means, gps_loc, N, times=[time2], 
                                          fig_path=fig_path, shape=(1,1))
plt.show()

In [ ]:
[round(val, 2) for val in distances[58]]

In [ ]:
distances, centroids = kmeans_utils.get_distances(means)

In [ ]:
all_time_points = kmeans_utils.get_centroid_circle_paths(distances, centroids)

In [ ]:
fig, ax = figure_functions.centroid_radius(centroids, all_time_points, gps_loc, times=range(6), 
                                               fig_path=fig_path, shape=(2,3))

# Train and Test GMM

In [ ]:
park_data, gps_loc, N = utils.load_daily_data(data_path)

In [ ]:
check = np.array(sorted(park_data['Load'].values.tolist(), reverse=True))

In [ ]:
len(np.where(check > 1.5)[0])/float(len(check)) * 100

In [ ]:
pd.DataFrame(range(10), index=range(10), columns=['check'])

In [ ]:
results = gmm.locational_demand_analysis(park_data, gps_loc, N)

In [ ]:
days = [result[0] for result in results]
hours = [result[1] for result in results]
errors = [result[2] for result in results]
morans_mix = [result[3] for result in results]
morans_adj = [result[4] for result in results]
means = [result[5] for result in results]

In [ ]:
scores, times = kmeans_utils.get_time_scores(means)

In [ ]:
best_times = {}
best_days = {}
for time in times:
    day = time/12
    hour = time%12 + 8
    if hour not in best_times:
        best_times[hour] = time
    if day not in best_days and hour not in [8, 15,16,17]:
        print day, hour
        best_days[day] = time

In [ ]:
good_times = []

for time in times:
    if time % 12 + 8 in [8,9]:
        continue
        
    good_times.append(time)
    data = np.vstack((means[time]))
    kmeans = KMeans(n_clusters=3).fit(data)
    labels = kmeans.labels_.tolist()

In [ ]:
good_times = best_days.values()

In [ ]:
fig, ax = figure_functions.centroid_plots(means, gps_loc, N, times=good_times, fig_path=fig_path, shape=(2,3))